### Plan of Attack
1. We will build a simple NN (single neuron(perceptron))

2. Train it on a real world dataset(dataset used: Breast Cancer dataset)

3. Will mimic the PyTorch workflow

4. Will have a lot of manual elements(making our own NN, loss func, gradient descent, etc)

5. End result is not important
(Main goal is not to improve accuracy of model but to understand the concept of how we build NN in PyTorch)

### Code Flow

1. Load the Dataset
2. Basic Preprocessing
3. Training Process
    
    a. Create the model
    
    b. Forward Pass
    
    c. Loss Calculation
    
    d. BackPropagation
    
    e. Parameters Update

4. Model Evaluation

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [5]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


#### Train Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

#### Scaling

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train

array([[ 2.52530338e+00,  1.26813347e-01,  2.46113814e+00, ...,
         1.75982043e+00,  9.39233383e-02, -6.16803515e-01],
       [-1.48070181e-02,  1.81775951e+00, -1.50926065e-02, ...,
        -1.12327560e-01, -1.06093757e+00,  1.51122777e-03],
       [ 9.47773906e-02, -9.47353275e-01,  9.05305431e-02, ...,
         7.35989500e-01, -2.86935046e-01, -3.76347782e-01],
       ...,
       [-6.61074044e-01, -3.97564767e-01, -6.46792447e-01, ...,
        -1.70832185e-01, -6.60773000e-01, -3.32520875e-01],
       [-7.98757019e-01,  1.56843812e-01, -7.35287519e-01, ...,
        -2.91844382e-01,  1.76413403e-01, -7.75527990e-01],
       [-6.94792324e-01,  2.31210716e+00, -6.88389209e-01, ...,
        -6.73510079e-01,  5.18007520e-02, -6.50562079e-01]])

In [9]:
y_train

,diagnosis
503,M
560,B
148,B
210,M
383,B
...,...
214,M
163,B
530,B
288,B


#### Label Encoding

In [10]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [11]:
y_train

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### Numpy Arrays To PyTorch Tensors

In [13]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [14]:
X_train_tensor.shape

# this means we have 30 features
# therefore our NN will have 30 weights and 1 bias

torch.Size([455, 30])

In [15]:
y_train_tensor.shape

torch.Size([455])

#### Defining The Model

In [17]:
class MySimpleNN():

  def __init__(self, X):
#                X.shape[1] - no of features(30), weights matrix will be 30 x 1
    self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True) # initializing weights
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True) # initializing bias

  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss



#### Important Parameters

In [16]:
learning_rate = 0.1
epochs = 25

#### Training Pipeline

In [20]:
# create model
model = MySimpleNN(X_train_tensor)

# define loop (whatever epochs we have defined it will run that many times)
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # backward pass
  loss.backward()

  # parameters update (weight and bias updation) (Wnew = Wold - lr(dl/dw))
  with torch.no_grad(): # disabling gradient tracking
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients (before another pass, clear gradient , so that gradient doesn't accumulate with each pass(they keep adding up))
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.6009252143536745
Epoch: 2, Loss: 3.476144854033568
Epoch: 3, Loss: 3.34920822851087
Epoch: 4, Loss: 3.2165034698010078
Epoch: 5, Loss: 3.0742635178646953
Epoch: 6, Loss: 2.928728284701491
Epoch: 7, Loss: 2.78029815262803
Epoch: 8, Loss: 2.6295370548112285
Epoch: 9, Loss: 2.475912848978295
Epoch: 10, Loss: 2.3269852619920917
Epoch: 11, Loss: 2.183510715907008
Epoch: 12, Loss: 2.041769155070878
Epoch: 13, Loss: 1.9027711805275564
Epoch: 14, Loss: 1.76527091910124
Epoch: 15, Loss: 1.6331844346418434
Epoch: 16, Loss: 1.508558712859651
Epoch: 17, Loss: 1.396568152000012
Epoch: 18, Loss: 1.296111528841575
Epoch: 19, Loss: 1.2047611373208964
Epoch: 20, Loss: 1.1277637112231411
Epoch: 21, Loss: 1.0642097822936598
Epoch: 22, Loss: 1.0125112287094513
Epoch: 23, Loss: 0.9706727196282365
Epoch: 24, Loss: 0.936668802753061
Epoch: 25, Loss: 0.9087279316589202


In [24]:
model.weights

tensor([[-0.2532],
        [-0.2217],
        [-0.2777],
        [-0.1747],
        [-0.2900],
        [-0.1662],
        [ 0.1822],
        [ 0.1915],
        [-0.2778],
        [ 0.2924],
        [ 0.4179],
        [ 0.5248],
        [ 0.4242],
        [-0.1637],
        [-0.1126],
        [ 0.0658],
        [-0.2523],
        [-0.3799],
        [ 0.7278],
        [ 0.2193],
        [-0.1179],
        [ 0.2067],
        [-0.3201],
        [ 0.4867],
        [ 0.2527],
        [ 0.2277],
        [-0.2506],
        [ 0.3471],
        [ 0.4832],
        [ 0.0982]], dtype=torch.float64, requires_grad=True)

In [25]:
model.bias

tensor([-0.1830], dtype=torch.float64, requires_grad=True)

#### Evaluation

In [27]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)

print(y_pred)

tensor([[0.4300],
        [0.7976],
        [0.0821],
        [0.3365],
        [0.7168],
        [0.2973],
        [0.0624],
        [0.1044],
        [0.0818],
        [0.2548],
        [0.5329],
        [0.1047],
        [0.1975],
        [0.2901],
        [0.1014],
        [0.3402],
        [0.2320],
        [0.5630],
        [0.2070],
        [0.6838],
        [0.4581],
        [0.7485],
        [0.5824],
        [0.5902],
        [0.8589],
        [0.5577],
        [0.7836],
        [0.9116],
        [0.9883],
        [0.1822],
        [0.9305],
        [0.0827],
        [0.6970],
        [0.1921],
        [0.6016],
        [0.2296],
        [0.7585],
        [0.2033],
        [0.7214],
        [0.9176],
        [0.2358],
        [0.4759],
        [0.9964],
        [0.8501],
        [0.5135],
        [0.7678],
        [0.8261],
        [0.1785],
        [0.2522],
        [0.7745],
        [0.5861],
        [0.3212],
        [0.2664],
        [0.8237],
        [0.4672],
        [0

In [ ]:
# our y_pred is in b/w 0 and 1 ( as sigmoid gives value b/w 0 to 1)
# but we need to find accuracy, we need to compare with original labels
# originals labels are either 0 or 1
# therefore we need to covert our y pred into 0s and 1s only
# therefore we can define a threshold (ex : 0.9)
# anything below 0.9 will be 0 anything above 0.9 will be 1

In [29]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()

print(y_pred)


tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [ ]:
# now we can find accuracy score b/w y_pred and y_test

In [30]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5132348537445068


In [ ]:
# Going forward we will improve this pipeline

# 1st improvement - we were building our own NN manually (deciding weight and bias)
# going forward we will us PyTorch nn module to create NN (can make complex NN easily)
# therefore we will replace our manually coded NN with a sophisticated NN module

# 2nd improvement - We coded our own loss function
# Pytorch gives us the feature to import any loss function easily and use it
# going forward rather than using manually coded loss func we will learn to use builtin PyTorch loss functions

# 3rd imporvement - instead of wrting manually coded gradient descent
# we will use optim class of pytorch , with its help we can apply gradient descent
# or any other optimizers like adam , rmsprop , etc by simply calling a function

# in this lecture we learned the framework, how does everything work
# but going forward, we will make changes to the framework and use pytorch elements
# rather than using our own manual elements.